In [1]:
import sys
sys.path.insert(0, "../../")
from pyecharts import online
online()
#  演示必要的准备代码，使用该库时不需重复此单元格命令

In [2]:
import xalpha as xa
import pandas as pd

# 多基金投资系统的管理

### 开放系统

In [3]:
path='../../tests/demo.csv'
read = xa.record(path) 
# 依旧先读取交易单的 csv 文件

In [4]:
sysopen = xa.mul(status=read.status) 
# 基金投资组合既可以由一系列单个基金的交易类生成， 也可以直接依据给定的交易单一次性生成，这里采取后者

In [5]:
sysopen.combsummary().sort_values(by="基金现值",ascending=False) # 先看一下今日投资组合概述

,基金名称,基金代码,当日净值,单位成本,持有份额,基金现值,基金总申购,历史最大占用,基金持有成本,基金分红与赎回,换手率,基金收益总额,投资收益率
5,总计,total,NaN,NaN,NaN,2545.38,4197.0,3775.09,2695.61,1501.39,0.366986,-150.23,-3.9795
2,富国中证红利指数增强,100032,1.0330,0.9969,1638.71,1692.79,1895.0,1633.66,1633.66,261.34,0.353730,59.13,3.6195
0,广发医药卫生联接A,001180,0.8090,0.8310,630.39,509.99,600.0,523.86,523.86,76.14,0.345889,-13.87,-2.6477
4,工银中证传媒指数分级,164818,0.9607,1.3460,356.62,342.60,480.0,480.00,480.00,0.00,0.267988,-137.40,-28.6250
1,广发中债7-10年国开债指数A,003376,0.9984,0.0000,0.00,0.00,742.0,742.00,-26.29,768.29,1.250728,26.29,3.5431
3,广发中证全指金融地产联接A,001469,0.8891,0.0000,0.00,0.00,480.0,480.00,84.38,395.62,1.381403,-84.38,-17.5792


In [6]:
sysopen.xirrrate() # 最重要的总是先问一下系统的整体收益年化如何
# 需要注意由于QDII基金的净值会延迟两天，因此包含 QDII 基金投资的 xirrrate 计算，必须指定日期，最晚为前天，默认的昨天计算会报错

-0.07499131908930949

In [7]:
sysopen.v_positions('2018-06-01') # 可视化指定日期的仓位分布

In [8]:
sysopen.v_positions_history() # 基金组合就像一条河流。。。

In [9]:
sysopen.v_tradevolume(bar_category_gap='75%',freq='W') 
# 整体交易情况一览, 以周为柱形图单位

In [10]:
sysopen.totcftable # 基金组合的合并现金流量表

,cash,date
0,-1170.00,2016-09-28
1,157.54,2017-01-17
2,-625.00,2017-10-17
3,-442.06,2017-12-12
4,-300.00,2018-01-18
5,-1395.57,2018-07-05
6,1079.48,2018-07-26


In [8]:
ev=sysopen.evaluation(start='2018-01-01') # 还可以生成关于投资基金的 evaluate 类，以便对投资的基金本身进行分析

In [9]:
ev.v_correlation(end='2018-08-01') #可以通过热力图来查看不同基金净值走势的相关性

### 封闭系统
我们也可以研究一个封闭系统的组合投资，此时需要货币基金提供资金出入的功能，这样的系统净值更容易定义
可以应用之前研究指数和基金时使用过的量化方法。

In [11]:
sysopen.fundtradeobj

(广发医药卫生联接A 交易情况,
 广发中债7-10年国开债指数A 交易情况,
 富国中证红利指数增强 交易情况,
 广发中证全指金融地产联接A 交易情况,
 工银中证传媒指数分级 交易情况)

In [10]:
# 这次我们使用基金交易类作为初始化参数
sysclose = xa.mulfix(*sysopen.fundtradeobj, totmoney=5000) 
# totmoney 表示封闭系统初始的总钱数
# 若不设置 cashobj 选项，则默认为 cashobj = cashinfo(interest=0.0001, start='2012-01-01')

In [21]:
sysclose.unitvalue('2018-07-04') # 封闭的基金投资组合整体就像一个单一基金交易类，有净值数据

1.016426

In [15]:
sysclose.combsummary('2018-03-09') # 投资组合的分项纵览

,基金名称,基金代码,当日净值,单位成本,持有份额,基金现值,基金总申购,历史最大占用,基金持有成本,基金分红与赎回,换手率,基金收益总额,投资收益率
0,广发医药卫生联接A,001180,0.876400,0.8134,275.20,241.19,300.00,300.00,223.86,76.14,0.434191,17.33,5.77670
1,广发中债7-10年国开债指数A,003376,0.940300,0.9424,575.14,540.80,542.00,542.00,542.00,0.00,1.276224,-1.20,-0.22140
2,富国中证红利指数增强,100032,1.178000,0.9843,877.47,1033.66,1125.00,863.66,863.66,261.34,0.555878,170.00,19.68370
3,广发中证全指金融地产联接A,001469,1.059300,1.0758,446.20,472.66,480.00,480.00,480.00,0.00,2.097701,-7.34,-1.52920
4,工银中证传媒指数分级,164818,0.829600,1.1980,225.38,186.98,270.00,270.00,270.00,0.00,0.346300,-83.02,-30.74810
5,货币基金,mf,1.253436,1.1647,2249.86,2820.06,3987.54,3987.54,2620.48,1367.06,0.465023,199.58,5.00510
6,总计,total,NaN,NaN,NaN,5295.35,6704.54,5000.00,5000.00,1704.54,0.346300,295.35,0.05907


In [16]:
sysclose.bcmkset(xa.indexinfo('0000905')) # 设置比较基准，开始对此封闭的基金投资系统进行量化评估

In [17]:
sysclose.v_netvalue()

In [18]:
sysclose.alpha(),sysclose.beta(),sysclose.sharpe()

(-0.002218844426589677, 0.19121239655826425, -0.79728321041646089)

In [19]:
sysclose.max_drawdown() # 系统的最大回撤

(Timestamp('2018-01-24 00:00:00'),
 Timestamp('2018-08-06 00:00:00'),
 -0.06905136122288996)

In [20]:
sysclose.riskfree  # 进行量化计算中使用的无风险年化利率的参考值， 可在设置 benchmark 时调整该值

0.0371724